# 크롤링 코드
피파인벤 홈페이지에서 포지션별로 유저 아아디, 선수이름, 댓글, 평점 데이터를 크롤링했다. 

In [1]:
import pandas as pd
import numpy as np
import requests
from requests.exceptions import HTTPError
from bs4 import BeautifulSoup

In [21]:
DATA_PATH = '.....'

In [33]:
#데이터를 저장하기 위한 빈 데이터 프레임 만들기
df = pd.DataFrame(columns = ['userId', 'player_name', 'position', 'comment', 'rating'])
df.head()

,userId,player_name,position,comment,rating


In [34]:
# 페이지와 포지션을 입력하여 댓글, 선수 이름, 평점, 유저 아이디를 크롤링 하는 함수
def crawling_func(pages, position):
    global df
    try:
        while pages <= 3800:
            url = f'https://fifaonline4.inven.co.kr/dataninfo/rate/index.php?pg={pages}&formation=' + position
            req = requests.get(url)
        
            soup = BeautifulSoup(req.content, 'html.parser')
            comment = soup.find_all(class_ = 'fifa4 comment')
            player_name = soup.find_all(class_ = 'fifa4 name')
            rating = soup.find_all(class_ = 'fifa4 rate')
            user_name = soup.find_all(class_ = 'text_right')

            for i in range(len(user_name)):
                user_id = user_name[i].get_text().split()[0]
                p_name =  player_name[i].get_text()
                cm = comment[i].get_text()
                r = rating[i].get_text()
                df = pd.concat([df, pd.DataFrame([[user_id,p_name, position ,cm, r]], columns = df.columns)])
                df.reset_index(inplace = True, drop = True)
            pages += 1
            if pages % 50 == 0:
                print(pages) # 100 pages 마다 출력

    except:
        pass
    df.to_csv(f'{DATA_PATH}fifa_comment_data_{position}.csv', index = False)
    return df    

In [ ]:
#공격수 포지션 크롤링
fw_df = crawling_func(1,'fw')
display(fw_df.head())
print(fw_df.shape)


In [47]:
df_fw = pd.read_csv(f'{DATA_PATH}fifa_comment_data_fw.csv')
print(df_fw.shape)
df_fw.tail()

(74940, 5)


,userId,player_name,position,comment,rating
74935,매찬세뭐임,네이마르 Jr.,fw,lh는 체감 쓰렉인데 체감이 같다니 뭔,4.13
74936,아재싸커,크리스티아누 호날두,fw,이거 5카 쓰느니 차라리 mc1카를 쓰셈 ㅋㅋ박스 안 슛= 맥두랑 주공두 비슷중거리...,3.92
74937,조세무리뉴,필리포 인자기,fw,너부터 인증하고 인증해달라고 하세요 가격값이 왜 세상에 없음? 그지라 비싼걸 써보지...,2.38
74938,명탐정피카42,데니스 베르캄프,fw,참고로 아이콘 가격과 벨캄 금카 가격이 차이가 안나요 그냥 벨캄 금카가 더이득!,4.38
74939,명탐정피카42,데니스 베르캄프,fw,TT 금카로 스트라이커에 투톱으로 쓰는중입니다. 벨캄 종결은 티티 금카인거같습니다....,4.38


In [48]:
#미드필더 포지션 크롤링
mf_df = crawling_func(1,'mf')
df_mf = pd.read_csv(f'{DATA_PATH}fifa_comment_data_mf.csv')
print(df_mf.shape)
df_mf.tail()

(78380, 5)


,userId,player_name,position,comment,rating
78375,한잔해,로타어 마테우스,mf,ㅉ 지가 못써놓고 지랄을,4.76
78376,애광이,파울로 디발라,mf,븅시나 15억에 팔았는데 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ싸물고 꺼지고,4.77
78377,플루토,크리스티안 에릭센,mf,토트넘이면 노미쓰고 인테르라면 LH쓰셈,4.37
78378,가성비갑선수,손흥민,mf,얘가 안좋다고요 ?손가락아프신가 ;??;;,4.21
78379,김승현호,호나우지뉴,mf,5카 개 좋음 묵직하면서 빠르고 감차랑 패스 지림 프리킥 간지는 덤,3.69


In [43]:
#수비수 포지션 크롤링
df_df = crawling_func(1,'df')
df_df = pd.read_csv(f'{DATA_PATH}fifa_comment_data_df.csv')
print(df_df.shape)
df_df.head()

(74920, 5)


,userId,player_name,position,comment,rating
0,단우홀,피카요 토모리,df,ebs 말디니 은카랑 얘 은카랑 같이 쓰는데 둘 다 속도가 빨라서 커서 잡고 수비좀...,3.90
1,강민경,피카요 토모리,df,은카 써봤고 지금 팔았음판 이유는 별거 아님성능은 좋음 진짜 만족그저 금카케미라서 ...,3.90
2,차니팬,뱅자맹 파바르,df,금카 삽니다 상한가에 올려놨어요 팔아주세여어어어,3.00
3,김재박,디에고 레예스,df,9카 팝니다 관심있으신 분 연락주세요,4.00
4,섹시몽주인,김광석,df,금카 후기같은 대한민국이라도 조금은 다르게?특별한 선수 쓰고싶어 찾다가 어렵게 28...,4.76


In [49]:
# 공격수, 미드필더, 수비수 데이터 합치기
df_all = pd.concat([df_fw,df_mf,df_df], axis = 0)
print(df_all.duplicated().sum())
df_all.drop_duplicates(inplace = True)
df_all.reset_index(inplace = True, drop = True)
df_all.shape

1867


(226373, 5)

In [50]:
print(df_all.duplicated().sum())
print(df_all.isnull().sum())

0
userId         0
player_name    0
position       0
comment        0
rating         0
dtype: int64


In [51]:
df_all.tail()

,userId,player_name,position,comment,rating
226368,다음달월클,부바카르 카마라,df,금카 삽니다 mevius0226 카톡주세요,0.00
226369,박주완개새,제이펏 탕강가,df,금카@구합니다 카톡 as1010101,3.75
226370,레알의라장군,카르바할,df,좋기만한데....,4.04
226371,아디동서남북,나우두,df,ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,3.67
226372,자기엘카,조 고메스,df,단점 헤딩 실존이랑 똑같음,3.83


In [52]:
df_all.to_csv(f'{DATA_PATH}fifa_comment_data.csv', index = False)